In [239]:
import pandas as pd
import numpy as np
from math import sqrt

In [349]:
ends = pd.read_csv('/Users/brentkong/Documents/curling/data/Ends.csv')
games = pd.read_csv('/Users/brentkong/Documents/curling/data/Games.csv')
stones = pd.read_csv('/Users/brentkong/Documents/curling/data/Stones.csv')

In [350]:
stones["GameUID"] = stones["CompetitionID"].astype(str) + "_" + \
                    stones["SessionID"].astype(str) + "_" + \
                    stones["GameID"].astype(str)

stones["TeamEndUID"]   = stones["CompetitionID"].astype(str) + "_" + \
                    stones["SessionID"].astype(str) + "_" + \
                    stones["GameID"].astype(str) + "_" + \
                    stones["TeamID"].astype(str) + "_" + \
                    stones["EndID"].astype(str)

stones["TeamEndThrowUID"]   = stones["CompetitionID"].astype(str) + "_" + \
                    stones["SessionID"].astype(str) + "_" + \
                    stones["GameID"].astype(str) + "_" + \
                    stones["TeamID"].astype(str) + "_" + \
                    stones["EndID"].astype(str) + "_" + \
                    stones["ShotID"].astype(str)

ends["GameUID"]   = ends["CompetitionID"].astype(str) + "_" + \
                    ends["SessionID"].astype(str) + "_" + \
                    ends["GameID"].astype(str)

ends["TeamEndUID"]   = ends["CompetitionID"].astype(str) + "_" + \
                    ends["SessionID"].astype(str) + "_" + \
                    ends["GameID"].astype(str) + "_" + \
                    ends["TeamID"].astype(str) + "_" + \
                    ends["EndID"].astype(str)

games["GameUID"]  = games["CompetitionID"].astype(str) + "_" + \
                    games["SessionID"].astype(str) + "_" + \
                    games["GameID"].astype(str)

games["TeamUID_1"]  = games["CompetitionID"].astype(str) + "_" + \
                    games["SessionID"].astype(str) + "_" + \
                    games["GameID"].astype(str) + "_" + \
                    games["TeamID1"].astype(str)

games["TeamUID_2"]  = games["CompetitionID"].astype(str) + "_" + \
                    games["SessionID"].astype(str) + "_" + \
                    games["GameID"].astype(str) + "_" + \
                    games["TeamID2"].astype(str)

In [351]:
def add_hammer_ends():
    map = {}

    for i, row in games.iterrows():
        team_1_id = row['TeamUID_1']
        team_2_id = row['TeamUID_2']
        
        hammer_first_team = team_2_id
        
        game_ends = ends[ends['GameUID'] == row['GameUID']].sort_values('EndID')
        
        hammer = hammer_first_team
        for _, end_row in game_ends.iterrows():
            end_id = end_row['EndID']
            
            map[f"{team_1_id}_{end_id}"] = (hammer == team_1_id)
            map[f"{team_2_id}_{end_id}"] = (hammer == team_2_id)
            
            hammer = team_1_id if hammer == team_2_id else team_2_id

    ends['Hammer'] = ends['TeamEndUID'].map(map)



def add_cumulative_ends():
    map = {}
    
    ends['TeamGameID'] = ends.apply(
        lambda row: f"{row['CompetitionID']}_{row['SessionID']}_{row['GameID']}_{row['TeamID']}",
        axis=1
    )
    
    for team_game_id in ends['TeamGameID'].unique():
        team_ends = ends[ends['TeamGameID'] == team_game_id].sort_values('EndID')
        
        cumulative = 0
        for idx, row in team_ends.iterrows():
            cumulative += row['Result']
            map[row['TeamEndUID']] = cumulative
    
    ends['Cumulative'] = ends['TeamEndUID'].map(map)
    
    ends.drop('TeamGameID', axis=1, inplace=True)

add_hammer_ends()
add_cumulative_ends()

In [353]:
ends

,CompetitionID,SessionID,GameID,TeamID,EndID,Result,PowerPlay,GameUID,TeamEndUID,Hammer,Cumulative
0,0,1,1,19,1,0,NaN,0_1_1,0_1_1_19_1,True,0
1,0,1,1,27,1,1,NaN,0_1_1,0_1_1_27_1,False,1
2,0,1,1,19,2,2,NaN,0_1_1,0_1_1_19_2,True,2
3,0,1,1,27,2,0,NaN,0_1_1,0_1_1_27_2,False,1
4,0,1,1,19,3,0,NaN,0_1_1,0_1_1_19_3,True,2
...,...,...,...,...,...,...,...,...,...,...,...
5269,24250026,49,1,24,5,1,NaN,24250026_49_1,24250026_49_1_24_5,False,5
5270,24250026,49,1,14,6,3,2.0,24250026_49_1,24250026_49_1_14_6,True,4
5271,24250026,49,1,24,6,0,NaN,24250026_49_1,24250026_49_1_24_6,False,5
5272,24250026,49,1,14,7,0,NaN,24250026_49_1,24250026_49_1_14_7,True,4


In [354]:
stones

,CompetitionID,SessionID,GameID,EndID,ShotID,TeamID,PlayerID,Task,Handle,Points,...,stone_9_y,stone_10_x,stone_10_y,stone_11_x,stone_11_y,stone_12_x,stone_12_y,GameUID,TeamEndUID,TeamEndThrowUID
0,0,1,1,1,7,19,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_7
1,0,1,1,1,8,27,1,0,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_27_1,0_1_1_27_1_8
2,0,1,1,1,9,19,2,0,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_9
3,0,1,1,1,16,27,2,0,0,0,...,1608.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_27_1,0_1_1_27_1_16
4,0,1,1,1,17,19,2,7,0,3,...,1608.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26365,24250026,49,1,7,18,24,2,9,0,4,...,2001.0,500.0,567.0,0.0,0.0,0.0,0.0,24250026_49_1,24250026_49_1_24_7,24250026_49_1_24_7_18
26366,24250026,49,1,7,19,14,2,0,0,3,...,2001.0,500.0,567.0,0.0,0.0,0.0,0.0,24250026_49_1,24250026_49_1_14_7,24250026_49_1_14_7_19
26367,24250026,49,1,7,20,24,2,6,0,4,...,2001.0,500.0,567.0,272.0,797.0,0.0,0.0,24250026_49_1,24250026_49_1_24_7,24250026_49_1_24_7_20
26368,24250026,49,1,7,21,14,1,0,0,4,...,2001.0,500.0,567.0,272.0,797.0,0.0,0.0,24250026_49_1,24250026_49_1_14_7,24250026_49_1_14_7_21


In [ ]:
from math import sqrt

def dist_to_button(x, y):
    return sqrt((x - 750)**2 + (y - 800)**2)

def lateral_offset(x):
    return abs(x - 750)

def add_guard_lateral_offset():
    

    for i in range(len(stones)):
        x1, y1 = stones["stone_1_x"].iloc[i], stones["stone_1_y"].iloc[i]
        x7, y7 = stones["stone_7_x"].iloc[i], stones["stone_7_y"].iloc[i]
        if dist_to_button(x7, y7) > dist_to_button(x1, y1): # this means 7 is the guard as it is farther than the house: 


add_guard_house_stones()


0_1_1_19_1_7 → stone_1_x: 750.0
0_1_1_27_1_7 → Not found
0_1_1_27_2_7 → stone_1_x: 751.0
0_1_1_19_2_7 → Not found
0_1_1_19_3_7 → stone_1_x: 750.0
0_1_1_27_3_7 → Not found
0_1_1_27_4_7 → stone_1_x: 750.0
0_1_1_19_4_7 → Not found
0_1_1_19_5_7 → stone_1_x: 750.0
0_1_1_27_5_7 → Not found
0_1_1_19_6_7 → stone_1_x: 1089.0
0_1_1_27_6_7 → Not found
0_1_1_27_7_7 → stone_1_x: 1150.0
0_1_1_19_7_7 → Not found
0_1_1_27_8_7 → stone_1_x: 750.0
0_1_1_19_8_7 → Not found
0_1_2_20_1_7 → stone_1_x: 750.0
0_1_2_46_1_7 → Not found
0_1_2_46_2_7 → stone_1_x: 750.0
0_1_2_20_2_7 → Not found
0_1_2_20_3_7 → stone_1_x: 750.0
0_1_2_46_3_7 → Not found
0_1_2_46_4_7 → stone_1_x: 750.0
0_1_2_20_4_7 → Not found
0_1_2_20_5_7 → stone_1_x: 750.0
0_1_2_46_5_7 → Not found
0_1_2_20_6_7 → stone_1_x: 350.0
0_1_2_46_6_7 → Not found
0_1_2_46_7_7 → stone_1_x: 577.0
0_1_2_20_7_7 → Not found
0_1_2_20_8_7 → stone_1_x: 689.0
0_1_2_46_8_7 → Not found
0_1_3_17_1_7 → stone_1_x: 750.0
0_1_3_22_1_7 → Not found
0_1_3_17_2_7 → stone_1_x: 750

In [341]:
def dist_to_button(x, y):
    return sqrt((x - 750)**2 + (y - 800)**2)

def add_guard_house_stones():
    TeamEndUIDs = set(stones["TeamEndUID"].tolist())
    for uid in TeamEndUIDs:
        id = uid + "_" + str(7)
        print(id, stones.loc[stones["TeamEndThrowUID"] == id, "stone_1_x"])
        #print("Stone 7", stones.loc[stones["TeamEndThrowUID"] == id, "stone_7_x"])
        
add_guard_house_stones()



24250026_16_1_16_5_7 Series([], Name: stone_1_x, dtype: float64)
24250026_3_5_14_4_7 Series([], Name: stone_1_x, dtype: float64)
24250026_4_2_17_8_7 Series([], Name: stone_1_x, dtype: float64)
22230015_2_3_18_4_7 4460    750.0
Name: stone_1_x, dtype: float64
23240026_45_2_46_8_7 18520    433.0
Name: stone_1_x, dtype: float64
24250026_1_5_19_1_7 19140    748.0
Name: stone_1_x, dtype: float64
22230015_1_3_10_1_7 Series([], Name: stone_1_x, dtype: float64)
22230015_13_4_46_5_7 8680    750.0
Name: stone_1_x, dtype: float64
0_10_2_43_6_7 Series([], Name: stone_1_x, dtype: float64)
24250026_8_1_30_8_7 Series([], Name: stone_1_x, dtype: float64)
24250026_17_4_15_3_7 Series([], Name: stone_1_x, dtype: float64)
24250026_10_5_51_6_7 Series([], Name: stone_1_x, dtype: float64)
24250026_17_5_14_5_7 Series([], Name: stone_1_x, dtype: float64)
22230015_16_2_51_6_7 Series([], Name: stone_1_x, dtype: float64)
23240026_46_1_17_3_7 18550    750.0
Name: stone_1_x, dtype: float64
22230015_7_4_44_4_7 Serie

In [337]:
stones.iloc[0:20, :]

,CompetitionID,SessionID,GameID,EndID,ShotID,TeamID,PlayerID,Task,Handle,Points,...,stone_9_y,stone_10_x,stone_10_y,stone_11_x,stone_11_y,stone_12_x,stone_12_y,GameUID,TeamEndUID,TeamEndThrowUID
0,0,1,1,1,7,19,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_7
1,0,1,1,1,8,27,1,0,1,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_27_1,0_1_1_27_1_8
2,0,1,1,1,9,19,2,0,0,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_9
3,0,1,1,1,16,27,2,0,0,0,...,1608.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_27_1,0_1_1_27_1_16
4,0,1,1,1,17,19,2,7,0,3,...,1608.0,0.0,0.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_17
5,0,1,1,1,18,27,2,6,1,0,...,1608.0,590.0,1164.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_27_1,0_1_1_27_1_18
6,0,1,1,1,19,19,2,0,1,2,...,1608.0,590.0,1164.0,0.0,0.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_19
7,0,1,1,1,20,27,2,0,1,4,...,1608.0,590.0,1164.0,671.0,850.0,0.0,0.0,0_1_1,0_1_1_27_1,0_1_1_27_1_20
8,0,1,1,1,21,19,1,2,0,1,...,1608.0,590.0,1164.0,671.0,850.0,0.0,0.0,0_1_1,0_1_1_19_1,0_1_1_19_1_21
9,0,1,1,1,22,27,1,0,0,0,...,1608.0,590.0,1164.0,671.0,850.0,1031.0,496.0,0_1_1,0_1_1_27_1,0_1_1_27_1_22


In [300]:
stones.iloc[20][11:35]

stone_1_x      750.0
stone_1_y     1916.0
stone_2_x      657.0
stone_2_y     1832.0
stone_3_x        0.0
stone_3_y        0.0
stone_4_x        0.0
stone_4_y        0.0
stone_5_x        0.0
stone_5_y        0.0
stone_6_x        0.0
stone_6_y        0.0
stone_7_x      750.0
stone_7_y      650.0
stone_8_x        0.0
stone_8_y        0.0
stone_9_x        0.0
stone_9_y        0.0
stone_10_x       0.0
stone_10_y       0.0
stone_11_x       0.0
stone_11_y       0.0
stone_12_x       0.0
stone_12_y       0.0
Name: 20, dtype: object

### Stone Geometry Features

In [250]:
stones.columns

Index(['CompetitionID', 'SessionID', 'GameID', 'EndID', 'ShotID', 'TeamID',
       'PlayerID', 'Task', 'Handle', 'Points', 'TimeOut', 'stone_1_x',
       'stone_1_y', 'stone_2_x', 'stone_2_y', 'stone_3_x', 'stone_3_y',
       'stone_4_x', 'stone_4_y', 'stone_5_x', 'stone_5_y', 'stone_6_x',
       'stone_6_y', 'stone_7_x', 'stone_7_y', 'stone_8_x', 'stone_8_y',
       'stone_9_x', 'stone_9_y', 'stone_10_x', 'stone_10_y', 'stone_11_x',
       'stone_11_y', 'stone_12_x', 'stone_12_y', 'GameUID'],
      dtype='object')

In [ ]:


def ideal_guard_error(x, side):
    ideal = 1200 if side == 1 else 300
    return abs(x - ideal)

def open_lane_proxy(df, side):
    corridor = 1200 if side == 1 else 300
    others = df[~df["Stone_1_x"].isin([0, 4095])]
    if others.empty:
        return 750
    return others["Stone_1_x"].apply(lambda x: abs(x - corridor)).min()

In [252]:
stones.iloc[:, 11:34]

,stone_1_x,stone_1_y,stone_2_x,stone_2_y,stone_3_x,stone_3_y,stone_4_x,stone_4_y,stone_5_x,stone_5_y,...,stone_7_y,stone_8_x,stone_8_y,stone_9_x,stone_9_y,stone_10_x,stone_10_y,stone_11_x,stone_11_y,stone_12_x
0,750.0,1916.0,4095.0,4095.0,0.0,0.0,0.0,0.0,0.0,0.0,...,650.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,750.0,1916.0,4095.0,4095.0,0.0,0.0,0.0,0.0,0.0,0.0,...,650.0,892.0,1232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,750.0,1916.0,4095.0,4095.0,694.0,756.0,0.0,0.0,0.0,0.0,...,650.0,892.0,1232.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,963.0,1657.0,4095.0,4095.0,694.0,756.0,0.0,0.0,0.0,0.0,...,650.0,892.0,1232.0,418.0,1608.0,0.0,0.0,0.0,0.0,0.0
4,963.0,1657.0,4095.0,4095.0,694.0,756.0,636.0,1080.0,0.0,0.0,...,650.0,4095.0,0.0,418.0,1608.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26365,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,0.0,0.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,0.0,0.0,0.0
26366,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,760.0,1152.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,0.0,0.0,0.0
26367,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,4095.0,0.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,272.0,797.0,0.0
26368,4095.0,0.0,4095.0,0.0,429.0,179.0,4095.0,0.0,4095.0,0.0,...,0.0,901.0,470.0,787.0,2001.0,500.0,567.0,272.0,797.0,0.0
